<b>Define environment variables</b>

To be used in future training steps.  Note that the BUCKET_NAME defined below must exist in the GCP project. 

In [1]:
%env BUCKET_NAME=ross-keras
%env LOCAL_JOB_DIR=local-training-output
%env JOB_NAME=keras_1102_job4
%env REGION=us-central1
%env MODEL_NAME=keras_wnd_model
%env MODEL_VERSION=v2

env: BUCKET_NAME=ross-keras
env: LOCAL_JOB_DIR=local-training-output
env: JOB_NAME=keras_1102_job4
env: REGION=us-central1
env: MODEL_NAME=keras_wnd_model
env: MODEL_VERSION=v2


<b>Perform training locally with default parameters</b>

Training detail will be written locally to the folder referenced in the job-dir parameter.

Note - creating the data will take some time as the MinMax normalizer needs to be fit over the 100 M plus training rows.

In [ ]:
!gcloud ai-platform local train \
  --package-path trainer \
  --module-name trainer.task \
  --job-dir $LOCAL_JOB_DIR \
  -- \
  --create-data=False 

<b>Perform training on AI Platform</b>

The training job can also be run on AI Platform.  Note that in order for AI Platform to be able to complete the training job, the "Google Cloud ML Engine Service Agent" service account must be granted Cloud Storage and BigQuery admin roles.

Important: A single training job (either locally or using AI Platform) must complete with the create-data flag set to true for the remainig functionality to compolete.

In [ ]:
!gcloud ai-platform jobs submit training $JOB_NAME \
  --package-path trainer/ \
  --module-name trainer.task \
  --region $REGION \
  --python-version 3.5 \
  --runtime-version 1.13 \
  --job-dir gs://${BUCKET_NAME}/keras-job-dir-${JOB_NAME} \
  -- \
  --create-data=False 

<b>Perform hyperparameter tuning on AI Platform</b>

Training detail will be written to Cloud Storage in the folder referenced in the job-dir parameter

In [2]:
!gcloud ai-platform jobs submit training ${JOB_NAME}_hpt \
  --config hptuning_config.yaml \
  --package-path trainer/ \
  --module-name trainer.task \
  --region $REGION \
  --python-version 3.5 \
  --runtime-version 1.13 \
  --job-dir gs://${BUCKET_NAME}/keras-job-dir-hpt

Job [keras_1101_job15_hpt] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe keras_1101_job15_hpt

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs keras_1101_job15_hpt
jobId: keras_1101_job15_hpt
state: QUEUED


<b>Complete training on AI Platform</b>

Now that hyperparameters have been tuned, perform deeper training with the optimal hyperparameters in place.  Note that we've explicitly increased the train-steps and num-epochs parameters in addition to the tuned hyperparameters.

In [11]:
!gcloud ai-platform jobs submit training $JOB_NAME \
  --package-path trainer/ \
  --module-name trainer.task \
  --region $REGION \
  --python-version 3.5 \
  --runtime-version 1.13 \
  --job-dir gs://${BUCKET_NAME}/keras-job-dir-${JOB_NAME} \
  -- \
  --num-deep-layers=2 \
  --first-deep-layer-size=30 \
  --first-wide-layer-size=1233 \
  --learning-rate=0.003 \
  --wide-scale-factor=0.094 \
  --train-batch-size=132 \
  --dropout-rate=0.4 \
  --train-steps=50000 \
  --num-epochs=100 

Job [keras_1102_job4] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe keras_1102_job4

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs keras_1102_job4
jobId: keras_1102_job4
state: QUEUED


<b>Host the trained model on AI Platform</b>

Because we're passing a list of numpy arrays and not a single numpy array as input for inference, we'll need to establish a custom prediction module.  

First, execute the setup script to create a distribution tarball

In [12]:
!python setup.py sdist --formats=gztar

running sdist
running egg_info
creating trainer.egg-info
writing requirements to trainer.egg-info/requires.txt
writing trainer.egg-info/PKG-INFO
writing top-level names to trainer.egg-info/top_level.txt
writing dependency_links to trainer.egg-info/dependency_links.txt
writing manifest file 'trainer.egg-info/SOURCES.txt'
reading manifest file 'trainer.egg-info/SOURCES.txt'
writing manifest file 'trainer.egg-info/SOURCES.txt'

running check


creating trainer-0.1
creating trainer-0.1/trainer
creating trainer-0.1/trainer.egg-info
making hard links in trainer-0.1...
hard linking predictor.py -> trainer-0.1
hard linking setup.py -> trainer-0.1
hard linking trainer/__init__.py -> trainer-0.1/trainer
hard linking trainer/create_data_func.py -> trainer-0.1/trainer
hard linking trainer/create_scaler_func.py -> trainer-0.1/trainer
hard linking trainer/model.py -> trainer-0.1/trainer
hard linking trainer/task.py -> trainer-0.1/trainer
hard linking trainer.egg-info/PKG-INFO -> trainer-0.1/trainer.

Copy the tarball over to Cloud Storage

In [13]:
!gsutil cp dist/trainer-0.1.tar.gz gs://${BUCKET_NAME}/staging-dir/trainer-0.1.tar.gz

Copying file://dist/trainer-0.1.tar.gz [Content-Type=application/x-tar]...
/ [1 files][  8.3 KiB/  8.3 KiB]                                                
Operation completed over 1 objects/8.3 KiB.                                      


Next, create a new model on AI Platform

In [ ]:
!gcloud ai-platform models create $MODEL_NAME --regions $REGION

Next we create new version using our trained model

In [11]:
!gcloud beta ai-platform versions create $MODEL_VERSION \
  --model $MODEL_NAME \
  --runtime-version 1.13 \
  --python-version 3.5 \
  --origin gs://${BUCKET_NAME}/keras-job-dir-${JOB_NAME} \
  --package-uris gs://${BUCKET_NAME}/staging-dir/trainer-0.1.tar.gz \
  --prediction-class predictor.MyPredictor

Creating version (this might take a few minutes)......done.


<b>Prepare a sample for inference</b>

Note that we are using the same preprocessing methods used for training.

In [16]:
!python create_sample.py

Using TensorFlow backend.
/Users/rossclaytor/Desktop/taxi-git/chicago-taxi/venv/lib/python3.7/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
Produced sample with label 660 seconds.


<b>Make an inference on a new sample.</b>

Pass the sample object to the model hosted in AI Platform to return a prediction.

In [17]:
!gcloud ai-platform predict \
  --model $MODEL_NAME \
  --version $MODEL_VERSION \
  --json-instances input_sample.json

{
  "predictions": "640.1120370952192"
}
